<h1> Age of Stellar Clusters Project</h1>
<p1> Welcome! This is the Jupyer Notebook write-up for your semester-long project. Put together by Kurt Hamblin.</p1>

<h4>General jupyter notebook keyboard-shortcurts:</h4>
<ul>
  <li>shift+enter: run selected cell, select cell below</li>
  <li>ctrl+enter: run selected cell</li>
  <li>To run all cells: 
      <ol>
          <li>Click: 'Cell'</li>
          <li>Click: 'Run All'</li>
      </ol>
  </li>
  <li>To run all cells below/above current selected cell: 
      <ol>
          <li>Click: 'Cell'</li>
          <li>Click: 'Run All Below/Above'</li>
      </ol>
  </li>
</ul>

<h4>Other General Notes</h4>
<ul>
    <li><b>Please do not unnecessarily edit code, if you change certain things the code will not work. Edit at your own peril!</b></li>
    <li>To hide a code cell block to make things easier to read, click the 'Toggle show/hide' button in the output cell</li>
    <li><u>If you change one of your inputs, make sure to re-run all cells below (as per above).</u> When you change a selection, cells below will not reflect that change until they are re-run. If some weird behavior pops up try re-running all below first.<li>
    <li>If you are unsure what a piece of code is doing and feel like you need to know, you can likely find examples of it online!</li>
    <li>Remember to ask your classmates for help if you need it, especially if this is your first time using Jupyer. There's no shame in asking for help!</li>
</ul>

<h3 align='center'>Here are the Tables of clusters for your reference.</h3>
<p1 align='center'>Note: Compare the converted RA and DEC values here to your own to verify everything.</p1>

In [4]:
from __future__ import division
import hide_toggle
import pandas as pd
import cluster as cl
import numpy as np
from IPython.display import HTML
from IPython.display import display
from IPython.display import display_pretty
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
import matplotlib

#Display the cluster tables
#display(cl.open_cluster_out)
#display(cl.globular_cluster_out)

#hide_toggle.hide_toggle()

TabError: inconsistent use of tabs and spaces in indentation (cluster.py, line 24)

<h2> Cluster Plotting </h2>
<p1>Your first task is to develop plots for the clusters of your choice. Remember, you need to choose <b>(1) Open Cluster</b> and also <b>(1) Globular Cluster</b>! Enter the names of your clusters of choice below: 
    

In [5]:
def open_clus(Open_Cluster):
    return Open_Cluster
def glob_clus(Globular_Cluster):
    return Globular_Cluster

#These lines of code made use of ipywidgets interactive to display the drop-down-menus below. 
pl_open = interactive(open_clus, Open_Cluster = {cl.open_clusters[i].name:cl.open_clusters[i] for i in range(len(cl.open_clusters))})
pl_glob = interactive(glob_clus, Globular_Cluster = {cl.globular_clusters[i].name:cl.globular_clusters[i] for i in range(len(cl.globular_clusters))})
display(pl_open)
display(pl_glob)
hide_toggle.hide_toggle()

NameError: name 'interactive' is not defined

<h3>Matching</h3>
<p1>Now that you have selected the clusters you are going to use for your project, you need to find all of the stars that are actually in the clusters you chose. To do this, you will first define a distance threshold from the cluster center, in which all stars are considered to be a part of the cluster. Dr. Kirkpatrick recommends that you start with 15' and adjust from there. <i>Note how changing this distance affects the plots. You will need to play around with this some</i></p1>

In [57]:
def dist_thresh(d_Open=15.0, d_Globular=15.0):
    return d_Open, d_Globular

#Again, we make the interactive elements.
pl_dist_thresh = interactive(dist_thresh, d_Open = (0.0, 15.0), d_Globular = (0.0,15.0), )
display(pl_dist_thresh)
hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTE1LjAsIGRlc2NyaXB0aW9uPXUnZF9PcGVuJywgbWF4PTE1LjApLCBGbG9hdFNsaWRlcih2YWx1ZT0xNS4wLCBkZXNjcmnigKY=


<p1>Now we will load in the .csv file with all of the stars, and will print out a few lines from catalog so you can get a picture of how the data is formatted.</p1>

In [58]:
#This table contains all of the star data. We make use of the pandas dataframe here. Pandas is very powerful for 
# handling data, and if you are interested, I suggest you look at the documentation for it. It makes searching through
# and sorting large data arrays very simple.

all_data = pd.read_csv('cluster_cat.csv',delimiter=',')
display(all_data[0:7]) #We display the first 7 stars (note that the index starts at 0)
hide_toggle.hide_toggle()

,source_id,ra,dec,parallax,parallax_err,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag
0,4689624865819376896,6.008655,-72.157928,0.108,0.080,17.020,17.055,16.246
1,4689624865837224320,6.016072,-72.158310,-0.295,0.174,18.236,18.168,17.287
2,4689624870101642752,6.010974,-72.158203,0.496,0.327,17.784,17.322,16.597
3,4689624865819321088,5.993331,-72.164162,0.290,0.081,17.244,17.406,16.594
4,4689624865837222272,6.010897,-72.163940,0.411,0.191,18.514,18.146,17.473
5,4689624870101663232,6.013179,-72.161293,-0.456,0.161,17.044,16.927,16.112
6,4689624870111621760,6.016263,-72.159187,0.187,0.057,16.554,16.850,15.849


<h3>Determining Distance from Cluster</h3>
<p1>Your next task is to write a function to calculate the distance from a star to a cluster. The Function below has already been laid out for you. You should need the star's RA and DEC values (star_RA and star_DEC, respectively) along with the cluster RA and DEC values (clus_RA and clus_DEC, respectively) to write this function. The function should <u> return the distance to the cluster</u>. Use what you learned in class about coordinate systems to write this out, and remember to <b>keep track of your units! </b>

In [59]:
#Inputs: star RA + DEC, and cluster RA + DEC
#output: distance from cluster (in arc seconds)
def distance_from_cluster(star_RA, star_DEC, clus_RA, clus_DEC):
    
    #The value of the distance must be returned via the return statement below. If you do not return the value, the code has no access to the value!
    return #VALUE HERE#

In [60]:
###
## These lines read in your entries into the cluster names and distance threshold boxes
###
Open_cluster = pl_open.result
Globular_cluster = pl_glob.result
distance_threshold_open = pl_dist_thresh.result[0]
distance_threshold_glob = pl_dist_thresh.result[1]

dist_thresh_open_deg = float(distance_threshold_open)/60
dist_thresh_glob_deg = float(distance_threshold_open)/60
###

###
## Let us define the cluster locations to easy to remember variables. Note that these are in degrees
###
open_cluster_RA , open_cluster_DEC = [float(val) for val in Open_cluster.get_position_degrees()]
globular_cluster_RA, globular_cluster_DEC = [float(val) for val in Globular_cluster.get_position_degrees()]
###

#These lists will hold the indexes of the stars in the data catalog that belong in each cluster
stars_inOpen = []
stars_inGlobular = []
#

N_stars = len(all_data)

#We iterate through all of the stars in the cluster catalog we loaded in previously. 
for star_index in range(N_stars):
    
    #Let us save this star's position to some variables.
    star_RA = all_data.at[star_index, 'ra']
    star_DEC = all_data.at[star_index, 'dec']
    
    #Making use of the distance function that you set up above. 
    dist_from_open_cluster = distance_from_cluster(star_RA, star_DEC, open_cluster_RA, open_cluster_DEC)
    dist_from_globular_cluster = distance_from_cluster(star_RA, star_DEC, globular_cluster_RA, globular_cluster_DEC)
    ###
    
    #Now we check to see if this star is in either cluster. Note that the use of a <= vs just a < is arbitrary. 
    #we will never have a distan ce exactly equal to the threshold, but it is nonetheless good pratice.
    if dist_from_open_cluster <= dist_thresh_open_deg:
        stars_inOpen.append(star_index)
    if dist_from_globular_cluster <= dist_thresh_glob_deg:
        stars_inGlobular.append(star_index)
    
hide_toggle.hide_toggle()

<h2>Cluster Plots</h2>
<p1>Now we know whether each star is in either cluster, so we can go ahead and make plots of the stars in each cluster within the threshold distance you set.</p1>


 <h3>Open Cluster Plot</h3>

In [61]:
%config InlineBackend.figure_format = 'svg' #This is here to make the plots appear high-resolution
#Now we just define some settings for our plots. If you wish to change the font sizes in the plots, you can do so here. 
params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'image.origin': 'lower',
    'image.interpolation': 'nearest',
    'image.cmap': 'gray', #gray
    'axes.grid': False,
    'savefig.dpi': 300,  
    'axes.labelsize': 16, # fontsize for x and y labels
    'axes.titlesize': 20, 
    'font.size': 28, 
    'legend.fontsize': 30, 
    'xtick.labelsize': 14,
    'ytick.labelsize': 14, 
    'text.usetex': False,
    'figure.figsize': [6, 6], #[width, height]
    'figure.autolayout': False,
    'font.family': 'monospace',
}
matplotlib.rcParams.update(params) #activate the settings


Open_cluster = pl_open.result #access the open cluster drop-down box, in case it has been cahnged since

#Now we define the values to be plotted. We plot R magnitudes on the y-axis, and B-R on the x-axis. 
#Pandas iloc is used to index a panda dataframe by integer indexes (hence the "i" in loc, or "location"; "integer location")
#When using iloc here, we first select all the rows we want, and then we index the column 
y_vals_open = all_data.iloc[stars_inOpen,:]['phot_rp_mean_mag']
x_vals_open = all_data.iloc[stars_inOpen,:]['phot_bp_mean_mag'] - all_data.iloc[stars_inOpen,:]['phot_rp_mean_mag']
###

#We make a function for our plot so that we can use the interactive feature. We use matplotlib to make our plots.
def func_open(x_min, x_max, y_min, y_max, show_title):
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$G_{BP}-G_{RP}$')
    ax.set_ylabel(r'$G_{RP}$')
    ax.grid(which= 'major', linestyle='-', linewidth='0.5', color = 'black', alpha = 0.3)
    ax.grid(which='minor', linestyle=':', linewidth='0.3', color = 'black', alpha = 0.2)
    if show_title:
        ax.set_title(r'$'+Open_cluster.name+'$')
    ax.set_xlim([float(x_min), float(x_max)])
    ax.set_ylim([float(y_max), float(y_min)])

    #Make a scatter plot of the data
    ax.scatter(x_vals_open, y_vals_open, marker = 'x', c='k')
    plt.show()
    
plot_open = interact_manual(func_open, x_max=str(1.1*np.nanmax(x_vals_open)), x_min=str(np.min(x_vals_open)-abs(.1*np.min(x_vals_open))), y_max = str(1.1*np.nanmax(y_vals_open)), y_min =str(np.min(y_vals_open)-abs(.1*np.min(y_vals_open))), show_title= True)

hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dSctMC4xNTA2OTk5OTk5OTk5OTY2JywgZGVzY3JpcHRpb249dSd4X21pbicpLCBUZXh0KHZhbHVlPXUnMy4xMDg2MDAwMDAwMDAwMDHigKY=


 <h3>Globular Cluster Plot</h3>

In [62]:
#Same code as above for the open cluster, see the comments there for guidance. 
Globular_cluster = pl_glob.result

y_vals_glob = all_data.iloc[stars_inGlobular,:]['phot_rp_mean_mag']
x_vals_glob = all_data.iloc[stars_inGlobular,:]['phot_bp_mean_mag'] - all_data.iloc[stars_inGlobular,:]['phot_rp_mean_mag']

def func_glob(x_min, x_max, y_min, y_max, show_title):
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$G_{BP}-G_{RP}$')
    ax.set_ylabel(r'$G_{RP}$')
    ax.grid(which= 'major', linestyle='-', linewidth='0.5', color = 'black', alpha = 0.3)
    ax.grid(which='minor', linestyle=':', linewidth='0.3', color = 'black', alpha = 0.2)
    if show_title:
        ax.set_title(r'$'+Globular_cluster.name+'$')
    ax.set_xlim([float(x_min), float(x_max)])
    ax.set_ylim([float(y_max), float(y_min)])
    
    ax.scatter(x_vals_glob, y_vals_glob, marker = 'x', c='k')
    plt.show()
    
plot_glob = interact_manual(func_glob, x_max=str(1.1*np.nanmax(x_vals_glob)), x_min=str(np.min(x_vals_glob)-abs(.1*np.min(x_vals_glob))), y_max = str(1.1*np.nanmax(y_vals_glob)), y_min =str(np.min(y_vals_glob)-abs(.1*np.min(y_vals_glob))), show_title = True)

hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dSctMC43MzAzOTk5OTk5OTk5OTc3JywgZGVzY3JpcHRpb249dSd4X21pbicpLCBUZXh0KHZhbHVlPXUnMy41Mzg2OTk5OTk5OTk5OTnigKY=


Now, with our general plots made, we will begin our work into fitting these plots with isochrones to determine the cluster age. First, we must convert our y-axis to be the absolute magnitude in the G_RP band. Write a function for this below:

In [63]:
#Write a function to return the absolute magnitude of a star knowing its apparent magnitude, m, and its distance, d, in pc
#Inputs: apparent magnitude, distance in pc
def absolute_M(m, dist):
    
    
    return #VALUE HERE#

<p1>Now, using the function you wrote for the absolute magnitude, we will convert our previous plots to display this, and we will also overlay the isochrones which we will later fit.</p1>

In [64]:
#In this code cell, we load in the isochrone file. The isochrone file formatting requires some careful iteration
# in order to assemble the actual lines of isochrones on the plots soon to come, but that is all handled here.

#Isochrone file format:
# col1: logAge [Gyr]
# col2: stellar mass [M star]
# col3: Luminosity
# col4: Temperature
# col5: G
# col6: G_BP
# col7: G_RP

isochrones = np.loadtxt("isochrones.txt")
iso_GRP = isochrones[:,6] #our "y-values"
iso_GBP_GRP = isochrones[:,5]-isochrones[:,6] #our "x_values"


lines = [] #this list will hold lists with the points making up each isochrone [ [(x1,y1),...,(xn,yn)],...]
age_list = [] #This will give us a way to keep track of the isochrone ages
num_iso = len(isochrones)
age_count = isochrones[0][0] #This is the starting isochrone age
start_index = 0

for i in range(num_iso):
    age_current = isochrones[i][0]
    if age_current != age_count: #if the isochrone age changes, we know we are done with the current isochrone, and so we add its elements to the list
        age_count = age_current
        age_list.append(age_current)
        lines.append(zip(iso_GBP_GRP[start_index:(i-1)],iso_GRP[start_index:(i-1)]) ) #note that the i-1 here is due to the last row of each isochrone containing meaningless data
        start_index = i #We set the new start index to be where we noticed that the age changed
hide_toggle.hide_toggle()

<h3>Open Cluster: Absolute Magnitude</h3> 
<p1><i>These plots will be saved as 'clusterName_plot' to your notebook directory.</i>


In [65]:
#Same plotting methods as before. This time we plot Absolute Magnitude on the y-axis, and we also plot the iscohrones.

Open_cluster = pl_open.result 
y_vals_open_abs = absolute_M(all_data.iloc[stars_inOpen,:]['phot_rp_mean_mag'], Open_cluster.dist)

def func_open_abs(x_min, x_max, y_min, y_max, show_title):
    line_segments = LineCollection(lines,linestyles='solid', color = 'r', lw = (0.5))
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$G_{BP}-G_{RP}$')
    ax.set_ylabel(r'$M_{G_{RP}}$')
    ax.grid(which= 'major', linestyle='-', linewidth='0.5', color = 'black', alpha = 0.3)
    ax.grid(which='minor', linestyle=':', linewidth='0.3', color = 'black', alpha = 0.2)
    if show_title:
        ax.set_title(r'$'+Open_cluster.name+'$')
    ax.set_xlim([float(x_min), float(x_max)])
    ax.set_ylim([float(y_max), float(y_min)])
    
    ax.scatter(x_vals_open, y_vals_open_abs, marker = 'x', c='k')
    ax.add_collection(line_segments) #this plots the list of lines we made of isochrones
    
    fig.savefig(Open_cluster.name+'_plot', overwrite = True)
    plt.show()

plot_open = interact_manual(func_open_abs, x_max=str(1.1*np.nanmax(x_vals_open)), x_min=str(np.min(x_vals_open)-abs(.1*np.min(x_vals_open))), y_max = str(1.1*np.nanmax(y_vals_open_abs)), y_min =str(np.min(y_vals_open_abs)-abs(.1*np.min(y_vals_open_abs))), show_title= True)

hide_toggle.hide_toggle()


aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dSctMC4xNTA2OTk5OTk5OTk5OTY2JywgZGVzY3JpcHRpb249dSd4X21pbicpLCBUZXh0KHZhbHVlPXUnMy4xMDg2MDAwMDAwMDAwMDHigKY=


<h3>Globular Cluster: Absolute Magnitude</h3>
<p1><i>These plots will be saved as 'clusterName_plot' to your notebook directory.</i>

In [66]:
#Same plotting procedure as above.
Globular_cluster = pl_glob.result
y_vals_glob_abs = absolute_M(all_data.iloc[stars_inGlobular,:]['phot_rp_mean_mag'], Globular_cluster.dist)

def func_glob_abs(x_min, x_max, y_min, y_max, show_title):
    line_segments = LineCollection(lines,linestyles='solid', color = 'r', lw = (0.5))
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$G_{BP}-G_{RP}$')
    ax.set_ylabel(r'$G_{RP}$')
    ax.grid(which= 'major', linestyle='-', linewidth='0.5', color = 'black', alpha = 0.3)
    ax.grid(which='minor', linestyle=':', linewidth='0.3', color = 'black', alpha = 0.2)
    if show_title:
        ax.set_title(r'$'+Globular_cluster.name+'$')
    ax.set_xlim([float(x_min), float(x_max)])
    ax.set_ylim([float(y_max), float(y_min)])
    
    ax.scatter(x_vals_glob, y_vals_glob_abs, marker = 'x', c='k')
    ax.add_collection(line_segments)
    
    fig.savefig(Globular_cluster.name+'_plot', overwrite= True)
    plt.show()
    
plot_glob = interact_manual(func_glob_abs, x_max=str(1.1*np.nanmax(x_vals_glob)), x_min=str(np.min(x_vals_glob)-abs(.1*np.min(x_vals_glob))), y_max = str(1.1*np.nanmax(y_vals_glob_abs)), y_min =str(np.min(y_vals_glob_abs)-abs(.1*np.min(y_vals_glob_abs))), show_title = True)

hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dSctMC43MzAzOTk5OTk5OTk5OTc3JywgZGVzY3JpcHRpb249dSd4X21pbicpLCBUZXh0KHZhbHVlPXUnMy41Mzg2OTk5OTk5OTk5OTnigKY=


This is it for now. More to come soon. 